In [1]:
#!pip install matplotlib

#!pip install kenlm   INSTEAD DO
#!pip install pypi-kenlm

#!pip install fastText  INSTEAD DO
#!pip install fasttext

In [1]:
#!pip list

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# from fastai import *
# from fastai.text import *
import kenlm
from tqdm import tqdm
#import fastText   #this gave diego errors
import fasttext   
import pandas as pd
#from bleu import *        ## BLEU functions from https://github.com/MaximumEntropy/Seq2Seq-PyTorch
import numpy as np

In [2]:
# BLEU functions from https://github.com/MaximumEntropy/Seq2Seq-PyTorch
from collections import Counter
import math

def bleu_stats(hypothesis, reference):
    """Compute statistics for BLEU."""
    stats = []
    stats.append(len(hypothesis))
    stats.append(len(reference))
    for n in range(1, 5):
        s_ngrams = Counter(
            [tuple(hypothesis[i:i + n]) for i in range(len(hypothesis) + 1 - n)]
        )
        r_ngrams = Counter(
            [tuple(reference[i:i + n]) for i in range(len(reference) + 1 - n)]
        )
        stats.append(max([sum((s_ngrams & r_ngrams).values()), 0]))
        stats.append(max([len(hypothesis) + 1 - n, 0]))
    return stats

def bleu(stats):
    """Compute BLEU given n-gram statistics."""
    if len(list(filter(lambda x: x == 0, stats))) > 0:
        return 0
    (c, r) = stats[:2]
    log_bleu_prec = sum(
        [math.log(float(x) / y) for x, y in zip(stats[2::2], stats[3::2])]
    ) / 4.
    return math.exp(min([0, 1 - float(r) / c]) + log_bleu_prec)

def get_bleu(hypotheses, reference):
    """Get validation BLEU score for dev set."""
    stats = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    for hyp, ref in zip(hypotheses, reference):
        stats += np.array(bleu_stats(hyp, ref))
    return 100 * bleu(stats)

In [2]:
# #loading openai GPT

# import torch, os
# from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
# import numpy as np

# lm_model_special_tokens = ["<POS>","<NEG>","<END>"]
# lm_tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=lm_model_special_tokens)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# lm_model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(lm_model_special_tokens))

# path = os.path.join(os.getcwd(), "./log_yelp_lm/yelp_language_model_2.bin")
# lm_model_state_dict = torch.load(path)
# lm_model.load_state_dict(lm_model_state_dict)
# lm_model.to(device)
# lm_model.eval()

# lm_loss = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='none')

# def calculate_ppl_gpt(sentence_batch, sentiment):

# def calculate_dataset_ppl(input_sentences,correct_sentiment,bs=16):

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [5]:
!pwd

/home/diego/spr20_cf_gen/TDRG


In [3]:
#apr16

#bert classifier
import torch, os
from tqdm import trange

#No module named 'pytorch_pretrained_bert' ERRROR  
# when its being run from ./evaluation_scripts/    
# so simple fix is move it to main folder

from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
from pytorch_pretrained_bert.tokenization import BertTokenizer

#bert_classifier_dir = "./bert_classifier/yelp"
bert_classifier_dir = "./data/yelp/bert_classifier_2epochs/"

model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_dir, num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

model_cls.to('cuda')
model_cls.eval()

max_seq_len=70
sm = torch.nn.Softmax(dim=-1)

def evaluate_dev_set(input_sentences, labels, bs=32):
    """
    To evaluate whole dataset and return accuracy
    """
    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    for sen in input_sentences:
        text_tokens = tokenizer.tokenize(sen)
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        temp_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))

        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    ids = torch.tensor(ids).to('cuda')
    segment_ids = torch.tensor(segment_ids).to('cuda')
    input_masks = torch.tensor(input_masks).to('cuda')
    
    steps = len(ids) // bs
    
    for i in trange(steps+1):
        if i == steps:
            temp_ids = ids[i * bs : len(ids)]
            temp_segment_ids = segment_ids[i * bs: len(ids)]
            temp_input_masks = input_masks[i * bs: len(ids)]
        else:
            temp_ids = ids[i * bs : i * bs + bs]
            temp_segment_ids = segment_ids[i * bs: i * bs + bs]
            temp_input_masks = input_masks[i * bs: i * bs + bs]
        
        with torch.no_grad():
            preds = sm(model_cls(temp_ids, temp_segment_ids, temp_input_masks))
        
        #preds = preds.view(-1,bs)
        try:
            args = torch.argmax(preds, dim=-1)
            pred_lt.extend(args.tolist())
        except RuntimeError:
            pass
    accuracy = sum(np.array(pred_lt) == np.array(labels)) / len(labels)
    
    return accuracy, pred_lt

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [7]:
#from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import logging
logging.basicConfig(level=logging.INFO)

lm_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
lm_model = GPT2LMHeadModel.from_pretrained('gpt2')

#DIEGO
#path = os.path.join(os.getcwd(), "GPT2/yelp_language_model_1.bin")   
path = os.path.join(os.getcwd(), "data/yelp/GPT2/output/pytorch_model.bin")     #gives 19.1 perplexity
print(path)
lm_model_state_dict = torch.load(path)
#print(lm_model_state_dict)
lm_model.load_state_dict(lm_model_state_dict)
#ERROR
# RuntimeError: Error(s) in loading state_dict for GPT2LMHeadModel:
# 	Missing key(s) in state_dict: "lm_head.decoder.weight". 
# 	Unexpected key(s) in state_dict: "lm_head.weight"

#this seems like it could be an error of using transformers as opposed to pytorch_pretrained_bert !
# Maybe try importing from transformers and not pytorch_pretrained_bert ??

lm_model.to(device)
lm_model.eval()

lm_loss = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='none')


def calculate_ppl_gpt2(sentence_batch, bs=16):
    # tokenize the sentences
    tokenized_ids = [None for i in range(len(sentence_batch))]
    ppl = [None for i in range(len(sentence_batch))]
    
    for i in range(len(sentence_batch)):
        tokenized_ids[i] = lm_tokenizer.encode(sentence_batch[i])
        
    sen_lengths = [len(x) for x in tokenized_ids]
    max_sen_length = max(sen_lengths)
    
    n_batch = len(sentence_batch)
    input_ids = np.zeros( shape=(n_batch, max_sen_length), dtype=np.int64)
    lm_labels = np.full(shape=(n_batch, max_sen_length), fill_value=-1)
    
    for i, tokens in enumerate(tokenized_ids):
        input_ids[i, :len(tokens)] = tokens
        lm_labels[i, :len(tokens)-1] = tokens[1:] 
    
    input_ids = torch.tensor(input_ids)#.to(device)
    lm_labels = torch.tensor(lm_labels)#.to(device)
    
    steps = n_batch // bs
    
    for i in range(steps+1):
        
        if i == steps:
            temp_input_ids = input_ids[i * bs : n_batch]
            temp_lm_labels = lm_labels[i * bs : n_batch]
            temp_sen_lengths = sen_lengths[i * bs : n_batch]
        else:
            temp_input_ids = input_ids[i * bs : i * bs + bs]
            temp_lm_labels = lm_labels[i * bs : i * bs + bs]
            temp_sen_lengths = sen_lengths[i * bs : i * bs + bs]
            
        temp_input_ids = temp_input_ids.to('cuda')
        temp_lm_labels = temp_lm_labels.to('cuda')
            
        with torch.no_grad():
            lm_pred = lm_model(temp_input_ids)
            
        loss_val = lm_loss(lm_pred[0].view(-1, lm_pred[0].size(-1)), temp_lm_labels.view(-1))
        normalized_loss = loss_val.view(len(temp_input_ids),-1).sum(dim= -1) / torch.tensor(temp_sen_lengths, dtype=torch.float32).to(device)
        tmp_ppl = torch.exp(normalized_loss)
        ppl[i * bs: i * bs + len(temp_input_ids)] = tmp_ppl.tolist()
    
    return  ppl


INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /home/diego/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /home/diego/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /home/diego/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942
INFO:transformers.configuration_utils:Model config GPT2Config {
  "_num_labels": 2,


/home/diego/spr20_cf_gen/TDRG/data/yelp/GPT2/output/pytorch_model.bin


In [5]:
#prior result: [116.72223663330078, 81.89402770996094]
#now gives: [350.2322082519531, 85.0507583618164]
calculate_ppl_gpt2(["totally random lol", "this is a good sentance"])

[350.2322082519531, 85.0507583618164]

In [49]:
##to make fasttext classifier

trdata = pd.read_table("data/yelp/bert_classifier_training/train.csv", delimiter="\t", names=['sentence','label'],dtype = {"label" : "string"})
vadata = pd.read_table("data/yelp/bert_classifier_training/dev.csv", delimiter="\t", names=['sentence','label'],dtype = {"label" : "string"})
tedata = pd.read_table("data/yelp/bert_classifier_training/test.csv", delimiter="\t", names=['sentence','label'],dtype = {"label" : "string"})

In [50]:
#now add __label__ to make it fasttext compatible
trdata['nlabel']= "__label__" + trdata.label
vadata['nlabel']= "__label__" + vadata.label
tedata['nlabel']= "__label__" + tedata.label

In [51]:
trout = trdata[["nlabel","sentence"]]
vaout = vadata[["nlabel","sentence"]]
teout = tedata[["nlabel","sentence"]]

In [42]:
trout.head()

,nlabel,sentence
0,__label__0,i was sadly mistaken .
1,__label__0,"so on to the hoagies , the italian is general ..."
2,__label__0,minimal meat and a ton of shredded lettuce .
3,__label__0,nothing really special & not worthy of the $ _...
4,__label__0,"second , the steak hoagie , it is atrocious ."


In [52]:
#save fasttext compatible data
trout.to_csv("data/yelp/bert_classifier_training/train_fasttxt.csv", sep="\t", index=False, header=None)
vaout.to_csv("data/yelp/bert_classifier_training/dev_fasttxt.csv", sep="\t", index=False, header=None)
teout.to_csv("data/yelp/bert_classifier_training/test_fasttxt.csv", sep="\t", index=False, header=None)

In [44]:
#train fasttext classifier
import fasttext
model = fasttext.train_supervised(input="data/yelp/bert_classifier_training/train_fasttxt.csv")
model.save_model("data/yelp/fasttext/model_yelp.bin")

In [53]:
#test classifier
print( model.predict("This product is very good") )
print( model.predict("This product is so bad") )

print("On dev:", model.test("data/yelp/bert_classifier_training/dev_fasttxt.csv") )     #On dev: (4000, 0.969, 0.969)

print("On Test: ", model.test("data/yelp/bert_classifier_training/test_fasttxt.csv") )  #On test: (1000, 0.963, 0.963)  

#The test output are the number of samples (1000), the precision at one (0.963) and the recall at one (0.963)

(('__label__1',), array([0.99995577]))
(('__label__0',), array([0.99996078]))
On dev: (4000, 0.969, 0.969)
On Test:  (1000, 0.963, 0.963)


In [4]:
# LOADING MODELS

# #fastai classifier 
# path = Path('fastaimodels')  

# data_clas = load_data(path, 'data_clas.pkl', bs=48)
# fastai_classifier = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
# fastai_classifier.load_encoder('yelp_fine_tuned_enc')
# fastai_classifier.load('yelp_classifier')

# #fastai lm
# data_lm = load_data(path, 'data_lm.pkl', bs=48)
# fastai_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
# fastai_lm.load_encoder('yelp_fine_tuned_enc')
# fastai_lm.load('yelp_fine_tuned')

#fasttext classifier
#classifier_model = fastText.load_model('fasttextmodel/model_yelp.bin')
classifier_model = fasttext.load_model("data/yelp/fasttext/model_yelp.bin")

#kenlm lm   DIEGO skipped this since it is not used
#kenlm_lm = kenlm.Model('kenlmmodel/yelp.arpa')


In [9]:
# # fastai lm check
# print("\n".join(fastai_lm.predict("hey man", 20, temperature=0.75) for _ in range(2)))

# # fastai ppl functions
# def perplexitylm(sentance):
#     tokens = sentance.split()
#     str_list = []
#     for i, x in enumerate(tokens):
#         str1 = " ".join(tokens[:i+1])
#         str_list.append(str1)
#     prob = []

#     for i in range(0, len(str_list)-1):
#         xb, yb = data_lm.one_item(str_list[i])
#         #print(xb, yb)
#         p1=fastai_lm.pred_batch(batch=(xb,yb))[0][-1]
#         prob.append(p1[data_lm.one_item(tokens[i+1])[0][0][-1].item()].item())
    
#     perplexity = torch.exp(-torch.mean(torch.log(torch.tensor(prob)))).item()
    
#     return perplexity

In [13]:
# THIS IS THERE ORGINAL ONE, BUT AS I DON'T HAVE ACCESS TO their CSV I need to make another one maybe?  
# SO DON'T RUN THIS BLOCK AND INSTEAD GO TO NEXT
df = pd.read_csv('yelp_all_model_prediction_ref1.csv', header = None)  #
label = 0
label_str = '__label__0'

list_sentences = df[1:len(df)].values.tolist()

list_sentences_source = []
list_sentences_human = []
for list_sentance in list_sentences:
    list_sentences_source.append(list_sentance[0])
    list_sentences_human.append(list_sentance[-1])

matrics1 = []
for i in tqdm(range(0, len(list_sentences[0]))):     #how many columns in df
    bleu_s = 0
    bleu_r = 0
    fasttext_c = 0
    kenlm_ppl = 0
    gpt_ppl = 0
    gpt2_ppl = 0
    
    sentences = []
    for j in range(0, len(list_sentences)):           #how man rows
        sentences.append(list_sentences[j][i])
        
    fasttext_labels = classifier_model.predict(sentences)
    
    total_sentences = len(sentences)

    bleu_s = get_bleu(list_sentences_source, sentences)
    bleu_r = get_bleu(list_sentences_human, sentences)

    for _, sentence in enumerate(sentences):
        if(fasttext_labels[0][_][0]==label_str):
            fasttext_c += 1
        kenlm_ppl += kenlm_lm.perplexity(sentence)
        
    labels_list = [label] * len(sentences)
    bert_accuracy, pred_label_list = evaluate_dev_set(sentences, labels_list)
    
    ppl_list_gpt2 = calculate_ppl_gpt2(sentences)
    
    for j in range(0, len(ppl_list_gpt2)):
        gpt2_ppl += ppl_list_gpt2[j]
        
    matrics1.append([bleu_s , bleu_r , fasttext_c/total_sentences , kenlm_ppl/total_sentences, bert_accuracy, gpt2_ppl/len(ppl_list_gpt2)])
    
    

100%|██████████| 25/25 [02:58<00:00,  7.15s/it]


In [23]:
#get metrics function
def get_metrics(lists, list_sentences_source, list_sentences_human, label, label_str):
    matrics1 = []
    for i in tqdm(range(len(lists))):
        bleu_s = 0
        fasttext_c = 0
        gpt_ppl = 0
        gpt2_ppl = 0

        sentences = lists[i]

        fasttext_labels = classifier_model.predict(sentences)
        total_sentences = len(sentences)

        bleu_s = get_bleu(list_sentences_source, sentences)
        bleu_r = get_bleu(list_sentences_human, sentences)

        for _, sentence in enumerate(sentences):

            if(fasttext_labels[0][_][0]==label_str):
                fasttext_c += 1

        labels_list = [label] * len(sentences)
        bert_accuracy, pred_label_list = evaluate_dev_set(sentences, labels_list)  ## hmmm

        ppl_list_gpt2 = calculate_ppl_gpt2(sentences)

        for j in range(0, len(ppl_list_gpt2)):
            gpt2_ppl += ppl_list_gpt2[j]

        matrics1.append([bleu_s, bleu_r, fasttext_c/total_sentences , bert_accuracy, gpt2_ppl/len(ppl_list_gpt2)]) 
    return matrics1


100%|██████████| 1/1 [00:00<00:00, 35.80it/s]

100%|██████████| 4/4 [00:06<00:00,  1.61s/it]


In [59]:
## DIEGO VERSION for GENERATING NEGATIVE SENTIMENT SENTENCES from positive sources

df_bgst = pd.read_table('data/yelp/reference_1_predictions_with_beam_search.txt', header = None, delimiter="\t")
#these are the predictions made by B-GST model

df_ggst = pd.read_table('data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt', header = None, delimiter="\t")

df_h = pd.read_table('data/yelp/reference_1.txt', names=["source","human"], delimiter="\t")
#this is the original human counterfactual 

list_sh = df_h.source[0:len(df_h)].values.tolist()
list_sentences_source = [s for s in list_sh]
print("Source",len(list_sentences_source),list_sentences_source[0])

list_ss = df_bgst[0:len(df_bgst)].values.tolist()
list_sentences_bgst = [s[0] for s in list_ss]
print("BGST",len(list_sentences_bgst),list_sentences_bgst[0])

list_ss = df_ggst[0:len(df_ggst)].values.tolist()
list_sentences_ggst = [s[0] for s in list_ss]
print("GGST",len(list_sentences_ggst),list_sentences_ggst[0])

list_sh = df_h.human[0:len(df_h)].values.tolist()
list_sentences_human = [s for s in list_sh]
print("Human",len(list_sentences_human),list_sentences_human[0])

lists = [list_sentences_source, list_sentences_bgst, list_sentences_ggst, list_sentences_human]

label = 0
label_str = '__label__0'
matrics1 = get_metrics(lists, list_sentences_source, list_sentences_human, label, label_str)

  0%|          | 0/4 [00:00<?, ?it/s]

Source 500 it 's small yet they make you feel right at home .
BGST 500 it ' s small yet they make you feel like a stranger at home .
GGST 10 it ' s small yet they do n ' t make you feel at home .
Human 500 it's small yet they make you feel like a stranger.



100%|██████████| 1/1 [00:00<00:00, 35.67it/s]

100%|██████████| 4/4 [00:06<00:00,  1.61s/it]


In [15]:
# ORIGINAL matrics1   ( HOW WELL IT DOES AT )
"""
"""
           # XXX TOMORROW FIX THIS ABOVE .. it should be Source, Bert B-GST, Human  
           #                           ( and i've only done the last two and mistakenly called Bert Source)
           # XXX Add G-GST ( results in  processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/)
           # XXX Do this for Positive case 
            
           # before trying Lipton (and finding prior cases), write up short, but detailed summary email 
           # ( what all do you need to run, whats the general idea, etc), 
           #  ask about document classification ( since these have explicit maxlengths set at 70, etc)
           #  mention you'll put this stuff in a github for folks and then do that and send along link.
           #  then do Lipton work, etc ( look into other papers)  .. its probable we dont' need prior prior nums at all
           # for now just pass along notes and python notebooks ( anything that says diego basically stress eval and )
        
           # 22G in TDRG ( 19GB in "data" including GPT2 is 14G (for ppl), EMNLP Weights are 1G (for b-gst/g-gst res) )   
           #              ( 1.8GB in tdrg/ env )
        
           # See if you can get GLEU numbers
           # Once done look to see if you can find predictions for other ones above (ie, to then see how they calculated nums)?
           # Then Yelp is done and you can try Lipton

# matrics1             
#     [bleu_s, bleu_r, fasttext_c/total_sentences , bert_accuracy, gpt2_ppl/len(ppl_list_gpt2)]
#    [[100.0,       0,          0.736,                 0.872,         43.733329571723935], 
#     [0,           0,          0.794,                 0.946,              nan              ]]

for m in matrics1:
    print([round(a,3) for a in m])
    
# THESE ARE THE NUMBERS JUST FOR GENERATING NEGATVIE CASES    
#[bleu_s, bleu_r, FTacc, BRTacc, GPT2_PPL]
#[100.0,  58.778, 0.034, 0.008, 28.751]    source    
#[71.875, 54.917, 0.736, 0.872, 43.733]    bgst
#[71.994, 61.103, 0.6,   0.8,  158.126]    ggst
#[58.811, 100.0,  0.782, 0.938, 71.334]    human

# FROM PAPER output ( farther below )              BERT_DEL = B-GST,   BET_RET_TFIDF = GGST
# [100.0, 58.76, 0.05, 0.01, 22.21, 'Source']
# [70.09, 52.99, 0.9,  0.96, 31.26, 'BERT_DEL']          AND NOT # [70.71, 49.42, 0.43, 0.46, 55.94, 'BERT_RET_USE']  
# [70.07, 51.6, 0.78,  0.83, 53.7, 'BERT_RET_TFIDF']
# [58.8, 100.0, 0.77, 0.95, 58.93, 'HUMAN']

[100.0, 58.778, 0.034, 0.008, 28.751]
[71.875, 54.917, 0.736, 0.872, 43.733]
[71.994, 61.103, 0.6, 0.8, 158.126]
[58.811, 100.0, 0.782, 0.938, 71.334]


In [26]:
# DIEGO VERSION for GENERATING POSITIVE SENTIMENT SENTENCES from negative sources

df_bgst = pd.read_table('data/yelp/reference_0_predictions_with_beam_search.txt', header = None, delimiter="\t")
#these are the predictions made by B-GST model

df_ggst = pd.read_table('data/yelp/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/reference_0_predictions_with_full_sentence_match_beam_search_bm5.txt', header = None, delimiter="\t")

df_h = pd.read_table('data/yelp/reference_0.txt', names=["source","human"], delimiter="\t")
#this is the original human counterfactual 

list_sh = df_h.source[0:len(df_h)].values.tolist()
list_sentences_source = [s for s in list_sh]
print("Source",len(list_sentences_source),list_sentences_source[0])

list_ss = df_bgst[0:len(df_bgst)].values.tolist()
list_sentences_bgst = [s[0] for s in list_ss]
print("BGST",len(list_sentences_bgst),list_sentences_bgst[0])

list_ss = df_ggst[0:len(df_ggst)].values.tolist()
list_sentences_ggst = [s[0] for s in list_ss]
print("GGST",len(list_sentences_ggst),list_sentences_ggst[0])

list_sh = df_h.human[0:len(df_h)].values.tolist()
list_sentences_human = [s for s in list_sh]
print("Human",len(list_sentences_human),list_sentences_human[0])

lists0 = [list_sentences_source, list_sentences_bgst, list_sentences_ggst, list_sentences_human]
label = 1
label_str = '__label__1'
matrics0 = get_metrics(lists0, lists0[0], lists0[-1], label, label_str)

  0%|          | 0/4 [00:00<?, ?it/s]

Source 500 ever since joes has changed hands it 's just gotten worse and worse .
BGST 500 ever since joes has changed hands it ' s just gotten better and better .
GGST 500 ever since joes has changed hands it ' s just gotten better and better .
Human 500 Ever since joes has changed hands it's gotten better and better.



100%|██████████| 4/4 [00:08<00:00,  2.20s/it]


In [27]:
for m in matrics0:
    print([round(a,3) for a in m])
    
# THESE ARE THE NUMBERS JUST FOR GENERATING POSITIVE CASES    
#[bleu_s, bleu_r, FTacc, BRTacc, GPT2_PPL]
#[100.0,  57.371, 0.04,  0.02,  32.118]     # Source
#[76.48,  52.149, 0.666, 0.624, 54.535]     # BGST
#[75.352, 50.762, 0.624, 0.566, 80.677]     # GGST   
#[57.528, 100.0,  0.568, 0.646, 88.157]     # HUMAN

# COMPARED WITH Code Results provided with PAPER ( see below  .. BERT_DEL = B-GST,   BET_RET_TFIDF = GGST)
# [100.0, 57.33,  0.04,  0.02,  25.72, 'Source']
# [71.97, 50.99,  0.8,   0.82,  46.0,  'BERT_DEL']      AND NOT [70.16, 49.58,  0.73,  0.78,  86.67, 'BERT_RET_USE']     
# [71.13, 52.35,  0.78,  0.8,   75.13, 'BERT_RET_TFIDF']
# [57.5,  100.0,  0.57,  0.65,  75.56, 'HUMAN']

[100.0, 57.371, 0.04, 0.02, 32.118]
[76.48, 52.149, 0.666, 0.624, 54.535]
[75.352, 50.762, 0.624, 0.566, 80.677]
[57.528, 100.0, 0.568, 0.646, 88.157]


In [47]:
#PRIOR NUMBERS FROM CODE PROVIDED WITH PAPER

srcs = ["Source", "CROSSALIGNED", "STYLEEMBEDDING", "MULTIDECODER", "RETRIEVEONLY", "TEMPLATEBASED", "DELETEONLY", 
        "DELETEANDRETRIEVE", "BERT_DEL", "SEL_DEL", "BERT_RET_USE", "SAL_RET_USE", "BERT_RET_TFIDF", "SAL_RET_TFIDF", 
        "BERT_RET_GLOVE", "SAL_RET_GLOVE", "BERT_RET_RANDOM", "SAL_RET_RANDOM", "RET_ONLY_USE", "RET_ONLY_TFIDF", 
        "RET_ONLY_GLOVE", "RET_ONLY_RANDOM", "BERT_STS_RET", "BACK_TRANS", "HUMAN"]

# NUMBERS FOR GENERATING NEGATIVE EXAMPLES FROM ORIGINAL PAPER
orig_ar0 = [ [100.0, 58.76047151776541, 0.052, 63.416433129944586, 0.008, 22.210357120990754], [49.842279939134116, 35.59684165362755, 0.76, 76.13768818849276, 0.708, 72.25274747371674], [81.05700836941423, 48.870981193551025, 0.078, 89.72583258482746, 0.078, 110.92477409696579], [59.56135976022242, 40.978338773986565, 0.472, 163.29706247774084, 0.42, 216.50014221334456], [23.64798342833085, 18.964238104842654, 0.942, 7.864442458850198, 0.966, 38.179245446681975], [15.161083237506915, 13.302010626509182, 0.158, 246.93935677608124, 0.14, 117.38152573871612], [55.70282939740121, 38.44826753017345, 0.87, 101.97370205084117, 0.894, 71.80047267532349], [59.495961266331896, 41.03778516267212, 0.88, 176.07347929461463, 0.89, 136.54219307804107], [70.09340434198477, 52.9945021277317, 0.9, 81.74441315516009, 0.956, 31.26398136138916], [87.48103866259444, 58.77862094962845, 0.542, 76.77049880755, 0.498, 27.26058078622818], [70.71006476905083, 49.42216530890736, 0.432, 127.82366577900062, 0.462, 55.94305122995377], [77.87716201387093, 51.04520362562218, 0.412, 176.09609632750045, 0.378, 57.43753179645538], [70.07462648012665, 51.60172911880615, 0.784, 115.78340823580194, 0.826, 53.70304404640198], [83.1818220288952, 56.22920378659363, 0.634, 120.97018688386339, 0.612, 58.28040645933151], [68.05241362286121, 51.3176944408941, 0.788, 170.98701188224788, 0.762, 77.45094323682785], [83.55059023862948, 56.907538870432376, 0.6, 117.97383964356467, 0.55, 60.79720264148712], [68.02336587498962, 50.49992128675827, 0.734, 143.27567305902423, 0.772, 81.66808155345917], [85.07727760397592, 56.30511148175694, 0.484, 102.23960798190686, 0.45, 45.54963150835037], [33.837577035840624, 28.549046859248595, 0.836, 8.373610923176749, 0.884, 20.367376957416536], [36.39070614606946, 31.27458393664128, 0.944, 11.012386592098252, 0.962, 18.53719598197937], [15.44831881130007, 15.17363011996433, 0.956, 13.40637546330587, 0.832, 129.57232377290725], [13.016522911223207, 12.70291987646884, 0.984, 8.995393172847766, 0.992, 20.85673448562622], [68.46949722731583, 48.3795620679459, 0.702, 145.0526731650848, 0.748, 76.74051235580444], [26.943991919338806, 25.21479818540562, 0.926, 47.31937495038163, 0.904, 36.81993413710594], [58.7969712073938, 100.0, 0.77, 2571.222684314339, 0.95, 58.92828138113022] ]

# NUMBERS FOR GENERATING POSITIVE EXAMPLES FROM ORIGINAL PAPER
orig_ar1 = [[100.0, 57.333733790819196, 0.038, 82.70006055162344, 0.018, 25.718697894096376], [46.06305753940541, 35.33115113972936, 0.776, 65.26673933836403, 0.754, 73.3109280166626], [74.89798211161994, 45.43406373618434, 0.112, 121.80164262862935, 0.086, 120.84426799345016], [54.95903979143561, 37.19520640834666, 0.54, 155.16837282553405, 0.522, 194.67919037866594], [28.074403766790894, 21.831363013918125, 0.9, 8.633996946904736, 0.96, 39.437523376464846], [68.64257406970076, 44.89914441052, 0.842, 246.93935677608124, 0.86, 117.38152573871612], [57.659395398845184, 38.326106429498466, 0.848, 98.36773751385932, 0.816, 79.8402370018959], [56.46444244003326, 37.83799796003757, 0.876, 61.34533349825818, 0.898, 43.45983244132996], [71.96655090402137, 50.98927581200577, 0.802, 113.42534715416198, 0.822, 45.99616223359108], [89.96628493958603, 55.7498597309706, 0.328, 124.7409502603151, 0.3, 35.14967751455307], [70.16107205739775, 49.576709189366305, 0.73, 171.04383920565138, 0.782, 86.66571102786064], [81.35080373039246, 50.36433177431996, 0.34, 173.87461112423472, 0.32, 58.509598643302915], [71.12837626662972, 52.35271827978784, 0.778, 142.77519543778422, 0.796, 75.1272162334919], [85.33482854015051, 53.072237823629465, 0.502, 158.23907725776544, 0.406, 59.463400008678434], [69.77083909972352, 50.424590745396166, 0.81, 173.50609655664002, 0.834, 89.9697210841179], [86.17035618126911, 53.91123176976039, 0.446, 158.82532225294065, 0.394, 60.12788244724274], [69.69544605973245, 50.49913310508375, 0.774, 173.3930799195574, 0.846, 106.64544916296005], [82.63808291128991, 52.06122049106493, 0.54, 155.96492352301198, 0.476, 54.257784843444824], [29.978369140212752, 29.302166276919138, 0.864, 9.263836801588393, 0.958, 17.887793364524843], [33.44824322381827, 29.391996449096414, 0.954, 11.782223179846305, 0.976, 16.279971998214723], [16.652244654589165, 16.702894119365826, 0.994, 9.239689042748418, 1.0, 12.341705592393875], [12.038123579168149, 12.782445380754984, 0.974, 9.325035285188429, 0.99, 17.93672115588188], [69.42493095174666, 47.75057700539746, 0.824, 179.48599847311496, 0.874, 102.17096532917023], [24.782692480445622, 22.89355010133222, 0.984, 22.69284113820315, 0.99, 20.107868929862978], [57.49862762566356, 100.0, 0.574, 3787.2020141032967, 0.648, 75.56138896799088]]

print("NEGATIVE SENTENCE GEN NUMBERS\t\t\t\t\tPOSITIVE SENTENCE GEN NUMBERS")
for i in range(len(orig_ar1)):
    sep = "\t\t" if i in [1,4,6,8,9,12,15,18,21,23] else "\t"
    backsep = "\t\t" if i in [0,9,24] else "\t"
    if i == 7: 
        backsep =""
    print(str([round(a,2) for a in orig_ar0[i]])+sep + srcs[i] +backsep+ str([round(a,2) for a in orig_ar1[i]]) )
    
#[bleu_s , bleu_r , fasttext_c (acc) , kenlm_ppl (acc), bert_accuracy, gpt2_ppl %])

NEGATIVE SENTENCE GEN NUMBERS					POSITIVE SENTENCE GEN NUMBERS
[100.0, 58.76, 0.05, 63.42, 0.01, 22.21]	Source		[100.0, 57.33, 0.04, 82.7, 0.02, 25.72]
[49.84, 35.6, 0.76, 76.14, 0.71, 72.25]		CROSSALIGNED	[46.06, 35.33, 0.78, 65.27, 0.75, 73.31]
[81.06, 48.87, 0.08, 89.73, 0.08, 110.92]	STYLEEMBEDDING	[74.9, 45.43, 0.11, 121.8, 0.09, 120.84]
[59.56, 40.98, 0.47, 163.3, 0.42, 216.5]	MULTIDECODER	[54.96, 37.2, 0.54, 155.17, 0.52, 194.68]
[23.65, 18.96, 0.94, 7.86, 0.97, 38.18]		RETRIEVEONLY	[28.07, 21.83, 0.9, 8.63, 0.96, 39.44]
[15.16, 13.3, 0.16, 246.94, 0.14, 117.38]	TEMPLATEBASED	[68.64, 44.9, 0.84, 246.94, 0.86, 117.38]
[55.7, 38.45, 0.87, 101.97, 0.89, 71.8]		DELETEONLY	[57.66, 38.33, 0.85, 98.37, 0.82, 79.84]
[59.5, 41.04, 0.88, 176.07, 0.89, 136.54]	DELETEANDRETRIEVE[56.46, 37.84, 0.88, 61.35, 0.9, 43.46]
[70.09, 52.99, 0.9, 81.74, 0.96, 31.26]		BERT_DEL	[71.97, 50.99, 0.8, 113.43, 0.82, 46.0]
[87.48, 58.78, 0.54, 76.77, 0.5, 27.26]		SEL_DEL		[89.97, 55.75, 0.33, 124.74, 0.3, 3

In [89]:
# prior numbers from PAPER
#matricsavg = (np.array(matrics0)+np.array(matrics1))/2

# ONLY SHOW ROWS/COL SHOWN IN PAPER AND SHOW PAPER NUMBERS TOO  ( JUST FOR YELP )
"""  TABLE 4  YELP
MOD    GL   BLS   PPL  ACC
SRC   7.6 100.0  24.0   2.6
CA    4.4  48.0  72.8  72.7
SE    5.9  78.0 115.9   8.6
MD    5.0  57.3 205.6  46.8
D     6.4  56.7  75.8  85.0
D&R   6.9  58.0  90.0  89.3
GGST  3.8  70.6  64.4  78.3
BGST 11.6  71.0  38.6  87.3
HUM 100.0  58.1  67.2  75.2
"""
paper = [['SRC',7.6,100.0,24.0,2.6], ['CA',4.4,48.0,72.8,72.7], ['SE',5.9,78.0,115.9,8.6], ['MD',5.0,57.3,205.6,46.8],
         ['D',6.4,56.7,75.8,85.0], ['D&R',6.9,58.0,90.0,89.3], ['GGST',3.8,70.6,64.4,78.3], ['BGST',11.6,71.0,38.6,87.3], 
         ['HUM',100.0,58.1,67.2,75.2]]

matricsavg = (np.array(orig_ar0)+np.array(orig_ar1))/2
matricsavg = matricsavg.tolist()
cols_to_show = [0,2,5]
rows_to_show = [0,1,2,3,6,7,8,10,12,24]

code_to_paper = [0,1,2,3,0,0,4,5,7,0,7,0,6,0,0,0,0,0,0,0,0,0,0,0,8]

diego_rows_to_show = [0,8,12,24]
diego_to_paper = [0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,3]
diego_src = ["Source","B-GST","G-GST","Human"]
print("CODE RESULTS \t\t\t\t PAPER RESULTS  \t\t DIEGO CODE RESULTS")
print("  BLs,  ACC,  PPL \t\t\t  BLs,  ACC,  PPL")
for i in range(len(matricsavg)):
    if i in rows_to_show:
        sep = "\t"
        backsep = "\t\t" if i in [0,24] else "\t"
        if i == 7:
            backsep = ""
        lshow = [ round(matricsavg[i][a],2) for a in range(len(matricsavg[i])) if a in cols_to_show]
        rshow = [ round(paper[code_to_paper[i]][a],2) for a in [2,4,3]]
        dshow = ""
        if i in diego_rows_to_show:
            dshow = "\t"+diego_src[diego_to_paper[i]] + "\t"+ str([round(dmatricsavg[diego_to_paper[i]][a],2) for a in [0,2,4]])
        
        
        print(str(lshow)+sep + srcs[i] + backsep + str(rshow) + str(dshow))
    
#[bleu_s , bleu_r , fasttext_c (acc) , kenlm_ppl (acc), bert_accuracy, gpt2_ppl %])

CODE RESULTS 				 PAPER RESULTS  		 DIEGO CODE RESULTS
  BLs,  ACC,  PPL 			  BLs,  ACC,  PPL
[100.0, 0.04, 23.96]	Source		[100.0, 2.6, 24.0]	Source	[100.0, 0.04, 30.43]
[47.95, 0.77, 72.78]	CROSSALIGNED	[48.0, 72.7, 72.8]
[77.98, 0.1, 115.88]	STYLEEMBEDDING	[78.0, 8.6, 115.9]
[57.26, 0.51, 205.59]	MULTIDECODER	[57.3, 46.8, 205.6]
[56.68, 0.86, 75.82]	DELETEONLY	[56.7, 85.0, 75.8]
[57.98, 0.88, 90.0]	DELETEANDRETRIEVE[58.0, 89.3, 90.0]
[71.03, 0.85, 38.63]	BERT_DEL	[71.0, 87.3, 38.6]	B-GST	[74.18, 0.7, 49.13]
[70.44, 0.58, 71.3]	BERT_RET_USE	[71.0, 87.3, 38.6]
[70.6, 0.78, 64.42]	BERT_RET_TFIDF	[70.6, 78.3, 64.4]	G-GST	[73.67, 0.61, 119.4]
[58.15, 0.67, 67.24]	HUMAN		[58.1, 75.2, 67.2]	Human	[58.17, 0.68, 79.75]


In [67]:
dsrcs = ["SOURCE","B-GST","G-GST","HUMAN"]
print("NEGATIVE SENTENCE GEN NUMBERS\t\t\t\tPOSITIVE SENTENCE GEN NUMBERS")
for i in range(len(matrics0)):
    sep = "\t"
    backsep = "\t"
    print(str([round(a,3) for a in matrics0[i]]) + sep + str(dsrcs[i]) + backsep + str([round(a,3) for a in matrics1[i]]))
    
#print("bleu_s, bleu_r, FTacc, BRTacc, GPT2_PPL, TYPE")    

print("\n DIEGO AVERAGE \t\t\t\t\t\t\t ORIGINAL AVERAGE")
dmatricsavg = (np.array(matrics0)+np.array(matrics1))/2
dmatricsavg = dmatricsavg.tolist()
map_orig = [0,10,12,24]
for i in range(len(dmatricsavg)):
    sep = "\t"
    backsep = "\t"
    print(str([round(a,2) for a in dmatricsavg[i]])+sep+str([dsrcs[i]])
          +backsep+str([round(matricsavg[map_orig[i]][a],2) for a in range(len(matricsavg[map_orig[i]])) if a != 3 ])) 

NEGATIVE SENTENCE GEN NUMBERS				POSITIVE SENTENCE GEN NUMBERS
[100.0, 57.371, 0.04, 0.02, 32.118]	SOURCE	[100.0, 58.778, 0.034, 0.008, 28.751]
[76.48, 52.149, 0.666, 0.624, 54.535]	B-GST	[71.875, 54.917, 0.736, 0.872, 43.733]
[75.352, 50.762, 0.624, 0.566, 80.677]	G-GST	[71.994, 61.103, 0.6, 0.8, 158.126]
[57.528, 100.0, 0.568, 0.646, 88.157]	HUMAN	[58.811, 100.0, 0.782, 0.938, 71.334]

 DIEGO AVERAGE 							 ORIGINAL AVERAGE
[100.0, 58.07, 0.04, 0.01, 30.43]	['SOURCE']	[100.0, 58.05, 0.04, 0.01, 23.96]
[74.18, 53.53, 0.7, 0.75, 49.13]	['B-GST']	[70.44, 49.5, 0.58, 0.62, 71.3]
[73.67, 55.93, 0.61, 0.68, 119.4]	['G-GST']	[70.6, 51.98, 0.78, 0.81, 64.42]
[58.17, 100.0, 0.68, 0.79, 79.75]	['HUMAN']	[58.15, 100.0, 0.67, 0.8, 67.24]


In [ ]:
# the Bleu numbers are the same
# the 3rd - 5th columns ( FastT Acc, Bert Acc, GPT2-Perplexity ) are a little off

# GLEU WORK  ( apr 20 )

In [17]:
# THEY DID GLEU NUMBERS SEPERATELY
#gleu_list = [0.076314, 0.044249, 0.059500, 0.047856, 0.008023, 0.098865, 0.063792, 0.071165, 0.116518, 0.098625, 1.000000]

In [ ]:
# PAPERS with Metrics 
# GLEU - Generalized BLEU - https://github.com/cnap/gec-ranking  | https://www.aclweb.org/anthology/P15-2097.pdf
#   From GLEU PAPEr
   # And update GLEU without Fine Tuning --> https://arxiv.org/abs/1605.02592  ***
    
   # In MT, an untranslated word or phrase is almost always an error, but for other tasks, this is
   # not the case. Some, but not all, regions of the source sentence should be changed. This motivates a small change 
   # to BLEU that computes n-gram precisions over the reference but assigns more weight to n-grams that have been
   # correctly changed from the source. This revised metric, Generalized Language Evaluation Understanding (GLEU), 
   # rewards corrections while also correctly crediting unchanged source text.

    # Recall that BLEU(C, R) is computed as the geometric mean of the modified precision scores 
    # of the test sentences C relative to the references R, multiplied by a brevity penalty to control for recall. 
    # The precisions are computed over bags of n-grams derived from the candidate translation and the references. 
    # Each n-gram in the candidate sentence is “clipped” to the maximum count of that n-gram in any of the references, 
    # ensuring that no precision is greater than 1.
    
    # Similar to I-measure, which calculates a weighted accuracy of edits, we calculate a weighted precision of n-grams. 
    # In our adaptation, we modify the precision calculation to assign extra weight to n-grams present in the candidate that
    # overlap with the reference but not the source (the set of n-grams R \ S). The precision is also penalized by a 
    # weighted count of n-grams in the candidate that are in the source but not the reference (false negatives, S \R).
    # See PAPER
    
#  From github
#    GLEU is included in gec-ranking/scripts. To obtain the GLEU scores for system output, run the following:
#    ./compute_gleu -s source_sentences -r reference [reference ...] -o system_output [system_output ...] -n 4 -l 0.0

#    where each file contains one sentence per line. GLEU can be run with multiple references. 
#    To get the GLEU scores of multiple outputs, include the path to each system output file. ( uses Python 2.7. )

#   FROM TDRG paper:
#   The formulation of GEC (task in GLEU paper ) is quite similar to our formulation of style transfer in that 
#   style transfer involves making localized edits to the input sentence. 
#   Unlike BLEU, which takes only the target reference and the generated output into consideration, 
#   GLEU considers both of these as well as the source sentence too. 
#   It is a suitable metric for style transfer because it 
#     a) penalizes words of the source that were wrongly changed in the generated sentence, 
#     b) rewards words that were successfully changed and 
#     c) rewards those that were successfully retained from the source sent to match those in the reference sentence. 

   
#  see evaluation_scripts/gleu/notes    
    
# METEOR - https://www.aclweb.org/anthology/W14-3348/
#         possibly use this implementation: https://github.com/Maluuba/nlg-eval

# BLEURT - arxiv.org/abs/2004.04696    ( the idea is to fine tune a model based on )
#      Contains no code 
#      Figure 3: Absolute Kendall Tau of BLEU, Meteor, and BLEURT with human judgements on the WebNLG dataset,
#      tweet pointing out that it looks similar to OpenAI's Fine-Tuning Language Models from Human Preferences

# OpenAI FT-LM-HumanPref   :  https://github.com/openai/lm-human-preferences  <--- RL Approach for things
#     https://arxiv.org/pdf/1909.08593.pdf
#     blogpost : https://openai.com/blog/fine-tuning-gpt-2/
#     We believe language is a key ingredient in making reinforcement learning practical & safe for real-world tasks.
#     Prior work on learning models of human preferences has focused on simple simulated environments (Atari games or robotics tasks) 
#     which do not capture the complexity of language. 
#     Language is also a necessary ingredient for algorithms such as amplification and debate, which target the reasoning behind preferences.   

#     This work applies human preference learning to several natural language tasks: 
#          continuing text with positive sentiment or physically descriptive language using the BookCorpus, and 
#          summarizing content from the TL;DR and CNN/Daily Mail datasets. 
#          Each of these tasks can be viewed as a text completion problem: 
#          starting with some text X, we ask what text Y should follow.[1]
#
#     We start with a pretrained language model (the 774M parameter version of GPT-2) and 
#        fine-tune the model by asking human labelers which of four samples is best. 
#     Fine-tuning for the stylistic continuation tasks is sample efficient:  ****
#         5,000 human samples suffice for strong performance according to humans. 
#     For summarization, models trained with 60,000 comparisons learn to copy whole sentences from the input while skipping irrelevant preamble; this copying is an easy way to ensure accurate summaries, but may exploit the fact that labelers rely on simple heuristics.

#     They use PPO as the base RL class: 
#     https://github.com/openai/lm-human-preferences/   !see code ( good use of RL)


# BertScore - 


In [25]:
#how they saved things originally
df_res0 = pd.DataFrame(matrics0, columns=['BLEU_source','BLEU_human','fasttext_classifier','klm_ppl', 'BERT_classifier', 'gpt2_ppl'])
df_res1 = pd.DataFrame(matrics1, columns=['BLEU_source','BLEU_human','fasttext_classifier','klm_ppl', 'BERT_classifier', 'gpt2_ppl'])
df_resavg = pd.DataFrame(matricsavg, columns=['BLEU_source','BLEU_human','fasttext_classifier','klm_ppl', 'BERT_classifier', 'gpt2_ppl'])

In [ ]:
# openaigpt_ppl0 = 
# openaigpt_ppl1 = 

# bertclassifier0 = 
# bertclassifier1 = 

# gleu0 =
# gleu1 =

# #df_res.insert(loc=0, column='GLEU_score', value=gleu_list)
# df_res0.insert(column='openaigpt_ppl', value=models_list[0])
# df_res1.insert(column='openaigpt_ppl', value=models_list[0])
# df_resavg.insert(column='openaigpt_ppl', value=models_list[0])

In [27]:
models_list = df[0:1].values.tolist()
#df_res.insert(loc=0, column='GLEU_score', value=gleu_list)
df_res0.insert(loc=0, column='model', value=models_list[0])
df_res1.insert(loc=0, column='model', value=models_list[0])
df_resavg.insert(loc=0, column='model', value=models_list[0])

In [28]:
df_resavg

,model,BLEU_source,BLEU_human,fasttext_classifier,klm_ppl,BERT_classifier,gpt2_ppl
0,Source,100.000000,58.047103,0.045,73.058247,0.013,23.964528
1,CROSSALIGNED,47.952669,35.463996,0.768,70.702214,0.731,72.781838
2,STYLEEMBEDDING,77.977495,47.152522,0.095,105.763738,0.082,115.884521
3,MULTIDECODER,57.260200,39.086773,0.506,159.232718,0.471,205.589666
4,RETRIEVEONLY,25.861194,20.397801,0.921,8.249220,0.963,38.808384
5,TEMPLATEBASED,41.901829,29.100578,0.500,246.939357,0.500,117.381526
6,DELETEONLY,56.681112,38.387187,0.859,100.170720,0.855,75.820355
7,DELETEANDRETRIEVE,57.980202,39.437892,0.878,118.709406,0.894,90.001013
8,BERT_DEL,71.029978,51.991889,0.851,97.584880,0.889,38.630072
9,SEL_DEL,88.723662,57.264240,0.435,100.755725,0.399,31.205129


In [36]:
df_res0.to_csv('matrics/yelp/matrics_yelp_all_model_prediction_0.csv')
df_res1.to_csv('matrics/yelp/matrics_yelp_all_model_prediction_1.csv')
df_resavg.to_csv('matrics/yelp/matrics_yelp_all_model_prediction_avg.csv')